## Compulsory Task 

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [24]:
# Import libraries
import numpy as np
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import fuzz, process
import chardet
import datetime

In [25]:
# Load up store_income_data.csv
store_income_df = pd.read_csv("store_income_data_task.csv")

In [26]:
store_income_df.head()

,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,United States/
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,Britain
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,United States
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,Britain/
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,United Kingdom


1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [27]:
countries = store_income_df.country.unique()
print(f"There are {len(countries)} unique countries")
countries

There are 77 unique countries


array(['United States/', 'Britain', ' United States', 'Britain/',
       ' United Kingdom', 'U.K.', 'SA ', 'U.K/', 'America',
       'United Kingdom', nan, 'united states', ' S.A.', 'England ', 'UK',
       'S.A./', 'ENGLAND', 'BRITAIN', 'U.K', 'U.K ', 'America/', 'SA.',
       'S.A. ', 'u.k', 'uk', ' ', 'UK.', 'England/', 'england',
       ' Britain', 'united states of america', 'UK/', 'SA/', 'SA',
       'England.', 'UNITED KINGDOM', 'America.', 'S.A..', 's.a.', ' U.K',
       ' United States of America', 'Britain ', 'England', ' SA',
       'United States of America.', 'United States of America/',
       'United States.', 's. africasouth africa', ' England',
       'United Kingdom ', 'United States of America ', ' UK',
       'united kingdom', 'AMERICA', 'America ',
       'UNITED STATES OF AMERICA', ' S. AfricaSouth Africa', 'america',
       'S. AFRICASOUTH AFRICA', 'Britain.', '/', 'United Kingdom.',
       'United States', ' America', 'UNITED STATES', 'sa',
       'United States

In [28]:
# Convert 'country' column to lowercase
store_income_df["country"]=store_income_df["country"].str.lower()

# Remove trailing white space in 'country' column
store_income_df["country"]=store_income_df["country"].str.strip()

countries = store_income_df.country.unique()
print(f"There are {len(countries)} countries")
countries

There are 37 countries


array(['united states/', 'britain', 'united states', 'britain/',
       'united kingdom', 'u.k.', 'sa', 'u.k/', 'america', nan, 's.a.',
       'england', 'uk', 's.a./', 'u.k', 'america/', 'sa.', '', 'uk.',
       'england/', 'united states of america', 'uk/', 'sa/', 'england.',
       'america.', 's.a..', 'united states of america.',
       'united states of america/', 'united states.',
       's. africasouth africa', 'britain.', '/', 'united kingdom.',
       's. africasouth africa/', 'united kingdom/',
       's. africasouth africa.', '.'], dtype=object)

2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [29]:
# Function to replace rows in the provided column of the provided DataFrame
# that match the provided string above the provided ratio with the provided string

def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = df[column].unique()
    
    # Get the top 13 closest matches from the input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=13, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # Only get matches with a ratio greater than 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # Get the rows of all the close matches in the dataframe
    rows_with_matches = df[column].isin(close_matches)

    # Replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    print("All done!")

In [30]:
replace_matches_in_column(df=store_income_df, column='country', string_to_match="united kingdom")
replace_matches_in_column(df=store_income_df, column='country', string_to_match="united states")
replace_matches_in_column(df=store_income_df, column='country', string_to_match="united states of america")
replace_matches_in_column(df=store_income_df, column='country', string_to_match="africasouth")
replace_matches_in_column(df=store_income_df, column='country', string_to_match="uk")
replace_matches_in_column(df=store_income_df, column='country', string_to_match="u.k")
replace_matches_in_column(df=store_income_df, column='country', string_to_match="sa")
replace_matches_in_column(df=store_income_df, column='country', string_to_match="s.a")
replace_matches_in_column(df=store_income_df, column='country', string_to_match="america")
replace_matches_in_column(df=store_income_df, column='country', string_to_match="britain")
replace_matches_in_column(df=store_income_df, column='country', string_to_match="england")

All done!
All done!
All done!
All done!
All done!
All done!
All done!
All done!
All done!
All done!
All done!


In [31]:
# Get all the unique values in the 'country' column
countries = store_income_df['country'].unique()

print(f"There are {len(countries)} unique countries")
countries

There are 17 unique countries


array(['united states', 'britain', 'united kingdom', 'u.k', 'sa',
       'america', nan, 's.a', 'england', 'uk', '',
       'united states of america', 's. africasouth africa', '/',
       's. africasouth africa/', 's. africasouth africa.', '.'],
      dtype=object)

In [32]:
# Replace remaining values manually
store_income_df.replace({'uk':'united kingdom',
                         'britain':'united kingdom',
                         'u.k':'united kingdom',
                         'england':'united kingdom',
                        'united states of america':'united states',
                         'america':'united states',
                         'sa':'south africa',
                         's.a':'south africa',
                         's. africasouth africa':'south africa',
                         's. africasouth africa/':'south africa',
                         's. africasouth africa.':'south africa',
                         '/':np.nan,
                         '':np.nan,
                         '.':np.nan
                        }, inplace=True)

# Get all the unique values in the 'country' column excluding empty cells
countries = store_income_df['country'].dropna().unique()

print(f"There are {len(countries)} unique countries")
countries

There are 3 unique countries


array(['united states', 'united kingdom', 'south africa'], dtype=object)

3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [33]:
# Create a column and convert data type for date_meeasured to 'datetime64'
store_income_df['date_parsed'] = pd.to_datetime(store_income_df['date_measured'], format='%d-%m-%Y')

# Create a column for todays date
store_income_df['todays_date'] = datetime.date.today()

# Create a column and convert data type for todays_date to 'datetime64'
store_income_df['todays_date_parsed'] = pd.to_datetime(store_income_df['todays_date'], format='%Y-%m-%d')

# Create a column to get the number of days that shows how many
# days ago it was measured from the current date.
store_income_df['days_ago'] = store_income_df['todays_date_parsed']-store_income_df['date_parsed']

store_income_df['days_ago'].head()

0    6474 days
1    6505 days
2    7350 days
3    6381 days
4   18541 days
Name: days_ago, dtype: timedelta64[ns]